# Actualizar PIP (Si es necesario)

In [ ]:
%pip install --upgrade pip

# Instalar las dependencias

In [3]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Importacion de dependencias

In [4]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from matplotlib import pyplot as plt  # type: ignore
from tensorflow.keras.layers import Dense  # type: ignore
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.losses import sparse_categorical_crossentropy  # type: ignore

## Importación de funciones 

In [ ]:
from labeling_images import prob_viz, mediapipe_detection, draw_landmarks, draw_styled_landmarks, extract_keypoints

# Verificacion de capacidad grafica 

In [6]:

print(tf.config.list_physical_devices("GPU"))
print("TensorFlow version:", tf.__version__)
print("Is GPU available:", tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.17.0
Is GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Creacion de las carpetas 

In [7]:
# Carpeta General
DATA_PATH = os.path.join("D_CORTO")
# Clases 
actions = np.array(["manzana", "coco"])
# Numero de subcarpetas
no_sequences = 2
# Cantidad de fotos por secuencia
sequence_length = 4

In [8]:
print(actions)

['manzana' 'coco']


Se crea la carpeta donde seran guardados los frames de cadad video
    

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

Captura de los frames, en este caso fotos de la seña que sean importante (4 frames) 

In [ ]:
# import cv2
# import os
# import numpy as np
# import mediapipe as mp


# # Inicializar la captura de video
# cap = cv2.VideoCapture(0)
# photo_count = {action: {str(sequence): 0 for sequence in range(no_sequences)} for action in actions}


# print("Presiona 'd' para tomar una foto. Presiona 'q' para salir.")

# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Hacer las detecciones y dibujar los puntos clave
#         image, results = mediapipe_detection(frame, holistic)
#         draw_styled_landmarks(image, results)

#         # Mostrar el video en una ventana
#         cv2.imshow('Video', image)

#         # Esperar a que se presione una tecla
#         key = cv2.waitKey(1) & 0xFF

#         if key == ord('q'):
#             break
#         elif key == ord('d'):
#              for action in actions:
#                 # Loop through sequences aka videos
#                 for sequence in range(no_sequences):#start_folder, start_folder+no_sequences):
#                     # Loop through video length aka sequence length
#                     for frame_num in range(sequence_length):

#                         # Read feed
#                         ret, frame = cap.read()
                                                        
#                         # Make detections
#                         image, results = mediapipe_detection(frame, holistic)

#                         # Draw landmarks
#                         draw_styled_landmarks(image, results)
                        
#                         # NEW Apply wait logic
#                         if frame_num == 0: 
#                             cv2.putText(image, 'STARTING COLLECTION', (120,200), 
#                                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
#                             cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                             # Show to screen
#                             cv2.imshow('OpenCV Feed', image)
#                             cv2.waitKey(500)
#                         else: 
#                             cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                             # Show to screen
#                             cv2.imshow('OpenCV Feed', image)
                        
#                         # NEW Export keypoints
#                         keypoints = extract_keypoints(results)
#                         npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                         np.save(npy_path, keypoints)
#                     # Show to screen
#                         cv2.imshow('OpenCV Feed', image)
#                         cv2.waitKey(500)
#                         photo_filename = f'{photo_count[action][str(sequence)]}.jpg'
#                         photo_path = os.path.join(DATA_PATH, action, str(sequence), photo_filename)
#                         cv2.imwrite(photo_path, image)
#                         photo_count[action][str(sequence)] += 1
#                         print(f'Foto guardada: {photo_path}')

#                         # Extraer y guardar los puntos clave
#                         keypoints = extract_keypoints(results)
#                         npy_path = os.path.join(DATA_PATH, action, str(sequence), f'{photo_count[action][str(sequence)] - 1}.npy')
#                         np.save(npy_path, keypoints)
#                         print(f'Puntos clave guardados: {npy_path}')
#                         break


# cap.release()
# cv2.destroyAllWindows()


In [ ]:
# Inicializar la captura de video
cap = cv2.VideoCapture(0)

# Variables de configuración
actions = ['coco', 'manzana']  # Lista de frutas
sub_folders = [str(i) for i in range(2)]  # Subcarpetas numeradas de 0 a 1
sequence_length = 4  # Número de fotos por secuencia

# Crear un diccionario para contar las fotos tomadas en cada secuencia
photo_count = {action: {sub_folder: 0 for sub_folder in sub_folders} for action in actions}
action_index = 0  # Índice para la fruta actual
subfolder_index = 0  # Índice para la sub carpeta actual
countdown = sequence_length  # Contador inicial para las fotos

print("Presiona 'd' para tomar una foto. Presiona 'q' para salir.")

with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Hacer las detecciones y dibujar los puntos clave
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        # Mostrar el video en una ventana
        cv2.putText(image, f'Fotos restantes: {countdown}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Video', image)

        cv2.putText(image, f'Carpeta manzana:{subfolder_index}:{actions[1]}', (10, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Video', image)

        

        # Esperar a que se presione una tecla
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
        elif key == ord('d'):
            # Tomar la foto y guardarla en formato JPG
            current_action = actions[action_index]
            current_subfolder = subfolders[subfolder_index]
            photo_filename = f'{photo_count[current_action][current_subfolder]}.jpg'
            photo_path = os.path.join(DATA_PATH, current_action, current_subfolder, photo_filename)
            cv2.imwrite(photo_path, image)
            photo_count[current_action][current_subfolder] += 1

            # Extraer y guardar los puntos clave en formato NPY
            keypoints = extract_keypoints(results)
            npy_filename = f'{photo_count[current_action][current_subfolder] - 1}.npy'
            npy_path = os.path.join(DATA_PATH, current_action, current_subfolder, npy_filename)
            np.save(npy_path, keypoints)

            # Actualizar el contador y mostrarlo en pantalla
            countdown -= 1
            cv2.putText(image, f'Foto guardada: {photo_filename}', (10, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.imshow('Video', image)

            if countdown == 0:
                # Mostrar mensaje de cambio de subcarpeta
                cv2.putText(image, 'Cambio de subcarpeta', (10, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('Video', image)
                cv2.waitKey(1000)

                

                # Cambiar a la siguiente subcarpeta
                subfolder_index += 1
                countdown = sequence_length  # Reiniciar el contador

                if subfolder_index >= len(subfolders):
                    subfolder_index = 0
                    action_index += 1

                    if action_index >= len(actions):
                        cv2.putText(image, 'Todas las fotos completadas', (10, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
                        cv2.imshow('Video', image)
                        cv2.waitKey(2000)
                        break

cap.release()
cv2.destroyAllWindows()